In [11]:
# Imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from tqdm import tqdm
import os
import pickle

# Load data
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

# Preprocessing
full_train_texts = train_df['text'].tolist() + dev_df['text'].tolist()
full_train_labels = train_df['label'].tolist() + dev_df['label'].tolist()

labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in train_df['label']]
y_dev = [label2id[label] for label in dev_df['label']]

# TF-IDF Vectorization
tfidf = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2,6),
    max_features=50000
)

tfidf.fit(full_train_texts)

X_train = tfidf.transform(train_df['text'])
X_dev = tfidf.transform(dev_df['text'])
X_test = tfidf.transform(test_df['text'])


# Too time consuming TODO: Try with cv before submitting
# # Hyperparameter Grid
# param_grid = {
#     'C': [0.01, 10],
#     'l1_ratio': [0.3, 0.7]
# }
# grid = list(ParameterGrid(param_grid))

# # Cache file
# CACHE_FILE = './cache/ENET_CV5_cache.pkl'

# # Load previous cache if exists
# if os.path.exists(CACHE_FILE):
#     with open(CACHE_FILE, 'rb') as f:
#         cache = pickle.load(f)
#     print(f"Loaded {len(cache)} cached results.")
# else:
#     cache = {}

# best_score = 0
# best_params = None

# # Define scoring
# scorer = make_scorer(f1_score, average='macro')

# # Search
# for params in tqdm(grid, desc="Grid Search with 5-Fold CV"):
#     param_key = tuple(sorted(params.items()))  # make a hashable key

#     if param_key in cache:
#         macro_f1 = cache[param_key]
#     else:
#         print(f'Start of parameter tuning: {param_key}')
#         model = LogisticRegression(
#             penalty='elasticnet',
#             solver='saga',
#             max_iter=1000,
#             random_state=42,
#             n_jobs=-1,
#             **params
#         )
#         # Perform 5-Fold CV here!
#         scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scorer, n_jobs=-1)
#         macro_f1 = np.mean(scores)

#         cache[param_key] = macro_f1

#         # Save cache every time
#         with open(CACHE_FILE, 'wb') as f:
#             pickle.dump(cache, f)
#         print(f'Cached {param_key}, f1: {macro_f1:.4f}')

#     if macro_f1 > best_score:
#         best_score = macro_f1
#         best_params = params

# print(f"Best parameters: {best_params}")
# print(f"Best Macro F1 Score (5-Fold CV): {best_score:.4f}")

# Train the best model fully
best_model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    C=1,
    l1_ratio= 0.3
    # **best_params
)
print("Fitting model...")
best_model.fit(X_train, y_train)

# Evaluate again on Dev Set
print("Evaluation model")
dev_preds = best_model.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score (retrained best model): {dev_macro_f1:.4f}")
# C=1, L1_ratio=0.7 -> 0.4172
# C=1, L1_ratio=0.5 -> 0.4222
# C=0.5, L1_ratio=0.5 -> 0.4024
# C=1, L1_ratio=0.3 -> 0.4261


Fitting model
Evaluation model
Validation Macro F1 Score (retrained best model): 0.4261


In [10]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # <-- Correct import here!
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from tqdm import tqdm
import numpy as np
import os

# Load Data
train_df = pd.read_csv('../data/train.csv')  # Replace with your path
dev_df = pd.read_csv('../data/dev.csv')

# PyTorch-ready dataset
class RedditDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts              # Store the list of comment texts
        self.labels = labels            # Store the corresponding list of integer labels
        self.tokenizer = tokenizer      # Store the tokenizer (e.g., RobertaTokenizer)
        self.max_length = max_length    # Store the maximum sequence length

    def __len__(self):
        return len(self.texts)  # Important: this allows DataLoader to know dataset size

    def __getitem__(self, idx):
        # Fetch the text and its corresponding label using the index
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize the text:
        # - truncation: cuts off texts longer than max_length
        # - padding: adds padding tokens to shorter texts
        # - return_tensors='pt': returns PyTorch tensors instead of lists
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length = self.max_length,
            return_tensors='pt' # stands for PyTorch
        )

        # Return a dictionary with model inputs and label
        return {
            'input_ids': encoding['input_ids'].squeeze(),         # Tensor of input token IDs .squeeze() again just removes the extra dimension.
            'attention_mask': encoding['attention_mask'].squeeze(), # Tensor indicating real tokens vs padding
            'labels': torch.tensor(label, dtype=torch.long)        # Make sure it becomes a PyTorch Tensor, because the model expects labels to be Tensors during training.
        }


### FUNCTIONS: train, train_MPT, evaluate
from torch.amp import GradScaler, autocast
# Use smaller (half-size) numbers for most calculations, so training is much faster and uses less memory — without losing much accuracy.
scaler = GradScaler()

# Mixed Precision Training
def train_MPT(model, loader):
    model.train()
    total_loss = 0
    for batch in tqdm(loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with autocast(device_type=device.type):  # 🔥 autocast for mixed precision
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation Function
def evaluate(model, loader):
    model.eval() # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    with torch.no_grad(): # Turns off gradient calculations (Saves memory and speeds up evaluation because we don't need gradients)
        for batch in tqdm(loader):
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Get predictions (pick the class with the highest score (probability) for each example in the batch)
            preds = torch.argmax(outputs.logits, dim=1) # outputs.logits = raw scores for each class
            # Save predictions and true labels
            # We Move predictions and labels from GPU to CPU and convert them into numpy arrays
            # because some Python operations (like NumPy or scikit-learn) cannot work directly with GPU Tensors.
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Take the F1 score for each label (class) separately, and then average them equally (It does not care if one label has 1000 examples and another class has 100 examples)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    return macro_f1

# -------------------------
# Data preprocessing
# -------------------------
# Encode labels to integers
labels = train_df['label'].unique()
label2id = {label: idx for idx, label in enumerate(labels)} # {"label": 'id'}
id2label = {idx: label for label, idx in label2id.items()} # {'id':'label}


train_df['label_id'] = train_df['label'].map(label2id)
dev_df['label_id'] = dev_df['label'].map(label2id)

# -------------------
# Prepare DataLoaders
# -------------------
# CONFIG

# Version 1
# MODEL_NAME = 'roberta-base'
# MAX_LENGTH = 128
# BATCH_SIZE = 64
# EPOCHS = 2
# LEARNING_RATE = 2e-5
# 1)Train Loss: 1.4018, Validation Macro F1: 0.4303 2)Train Loss: 1.2615, Validation Macro F1: 0.4485

# Version 2
MODEL_NAME = 'roberta-base'
MAX_LENGTH = 256
BATCH_SIZE = 32
EPOCHS = 4
LEARNING_RATE = 2e-5
# 1)Train Loss: 1.3989, Validation Macro F1: 0.4421 2)Train Loss: 1.2495, Validation Macro F1: 0.4591
# 3)Train Loss: 1.1111, Validation Macro F1: 0.4551 4)Train Loss: 0.9832, Validation Macro F1: 0.4615

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

# Prepare dataset for PyTorch (processes the raw data)
train_dataset = RedditDataset(train_df['text'].tolist(), train_df['label_id'].tolist(), tokenizer, MAX_LENGTH)
dev_dataset = RedditDataset(dev_df['text'].tolist(), dev_df['label_id'].tolist(), tokenizer, MAX_LENGTH)

# Create small batches (batches them without changing the data itself)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE)

# Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda is faster but it requires GPU
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
model.to(device) # Moves all model weights and computations to the device you selected (cuda if GPU, otherwise cpu)

# Optimizer and Scheduler
# model.parameters() are all the learnable weights in the model.
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE) # A type of optimizer that updates the model's weights to minimize the loss.
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps) # It controls how the learning rate changes during training. In our case it linearly decreases


MODEL_SAVE_PATH = f'cache/best_{MODEL_NAME}_model_{MAX_LENGTH}-{BATCH_SIZE}-{EPOCHS}-{LEARNING_RATE}.pt'

# Check if saved model exists
if os.path.exists(MODEL_SAVE_PATH):
    print(f"Loading saved model from {MODEL_SAVE_PATH}...")
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
else:
    print("No saved model found. Starting training...")

    # Run Training
    best_f1 = 0
    best_model_state = None

    for epoch in range(EPOCHS): # One epoch = One full pass through the entire training dataset.
        print(f"Epoch {epoch+1}/{EPOCHS}")
        train_loss = train_MPT(model, train_loader)
        val_f1 = evaluate(model, dev_loader)
        print(f"Train Loss: {train_loss:.4f}, Validation Macro F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    print(f'Best f1 for this model: {best_f1}')

    # After training, save the best model
    torch.save(best_model_state, MODEL_SAVE_PATH)
    print(f"Training finished. Best model saved to {MODEL_SAVE_PATH}")

    # Loads back the best version of the model that you saved during training (Forget your current weights — load these saved weights instead)
    model.load_state_dict(best_model_state)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading saved model from cache/best_roberta-base_model_256-32-4-2e-05.pt...


In [13]:
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score

# Prepare Inputs for Meta Model
# Elastic Net (TF-IDF) model outputs: dev_preds_enet_proba
# RoBERTa model outputs: dev_logits_roberta

# Get probabilities from Elastic Net (already using predict_proba)
dev_preds_enet_proba = best_model.predict_proba(X_dev)  # shape: (n_samples, n_classes)

# Get logits from RoBERTa
model.eval()
all_logits = []
with torch.no_grad():
    for batch in dev_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.cpu().numpy()
        all_logits.append(logits)

dev_logits_roberta = np.vstack(all_logits)  # shape: (n_samples, n_classes)

# Stack Features for Meta Model
X_dev_stack = np.hstack([dev_preds_enet_proba, dev_logits_roberta])
y_dev_array = np.array(y_dev)  # true labels for dev set

# Train Meta Model
meta_model = LogisticRegressionCV(cv=5, max_iter=1000, multi_class='multinomial')
meta_model.fit(X_dev_stack, y_dev_array)

# Predict and Evaluate
meta_preds = meta_model.predict(X_dev_stack)
stacked_f1 = f1_score(y_dev_array, meta_preds, average='macro')
print(f"Meta-model Validation Macro F1 Score: {stacked_f1:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1908: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Meta-model Validation Macro F1 Score: 0.4643


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification

import joblib
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from your_dataset_module import RedditDataset  # replace with actual import
from your_model_utils import collate_fn         # if applicable

# -------------------------------
# Config
# -------------------------------
MODEL_NAME = 'roberta-base'
MAX_LENGTH = 256
BATCH_SIZE = 32
MODEL_SAVE_PATH = f'cache/best_{MODEL_NAME}_model_{MAX_LENGTH}-{BATCH_SIZE}-4-2e-05.pt'
SAVE_DIR = "saved_models"

# -------------------------------
# Load data
# -------------------------------
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')

with open(os.path.join(SAVE_DIR, "label_maps.pkl"), "rb") as f:
    label_maps = pickle.load(f)
    label2id = label_maps['label2id']
    id2label = label_maps['id2label']

y_dev = dev_df['label'].map(label2id).values

# -------------------------------
# Load TF-IDF + Classic Models
# -------------------------------
with open(os.path.join(SAVE_DIR, "tfidf_vectorizer.pkl"), "rb") as f:
    tfidf = pickle.load(f)

X_dev = tfidf.transform(dev_df['text'])

logreg = joblib.load(os.path.join(SAVE_DIR, "logreg_model.pkl"))
xgb_clf = joblib.load(os.path.join(SAVE_DIR, "xgb_model.pkl"))

logreg_dev_probas = logreg.predict_proba(X_dev)
xgb_dev_probas = xgb_clf.predict_proba(X_dev)

# -------------------------------
# Load and Run RoBERTa
# -------------------------------
print("Loading RoBERTa model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label2id))
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.to(device)
model.eval()

def get_roberta_dev_probas(model, texts):
    dataset = RedditDataset(texts, None, tokenizer, MAX_LENGTH, inference=True)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE)
    all_probs = []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)
            all_probs.append(probs.cpu().numpy())

    return np.vstack(all_probs)

print("Generating RoBERTa dev probabilities...")
roberta_dev_probas = get_roberta_dev_probas(model, dev_df['text'].tolist())

# -------------------------------
# Meta Features + Meta-Model
# -------------------------------
X_meta_dev = np.hstack([logreg_dev_probas, xgb_dev_probas, roberta_dev_probas])

meta_model = LogisticRegression(max_iter=1000, random_state=42)
meta_model.fit(X_meta_dev, y_dev)

# Evaluate
meta_dev_preds = meta_model.predict(X_meta_dev)
meta_f1 = f1_score(y_dev, meta_dev_preds, average='macro')
print(f"STACKED DEV MACRO F1: {meta_f1:.4f}")

# Save final meta-model
joblib.dump(meta_model, os.path.join(SAVE_DIR, "meta_model.pkl"))
print("Meta-model saved.")
